In [1]:
import pandas as pd

from src.config import TRAIN_SET_PATH, EVAL_SET_PATH

train_set = pd.read_csv(TRAIN_SET_PATH)
eval_set = pd.read_csv(EVAL_SET_PATH)

In [4]:
len(train_set["subject_id"].unique()),len(eval_set["subject_id"].unique()) 

(72, 18)

In [6]:
train_set["group"].value_counts(normalize=True)

group
MCI    0.569444
HC     0.430556
Name: proportion, dtype: float64

In [7]:
eval_set["group"].value_counts(normalize=True)

group
MCI    0.555556
HC     0.444444
Name: proportion, dtype: float64